# Scanning the BP dataset with pysegy

This notebook demonstrates how to download the BP 1994 2-D seismic dataset, scan it using the `pysegy` utilities and then visualise the source and receiver positions as well as a few shot gathers.

In [ ]:
import sys
sys.path.append('..')  # allow importing the local pysegy package
import pysegy
import urllib.request, gzip, os, shutil
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
bp_url = 'http://s3.amazonaws.com/open.source.geoscience/open_data/bpmodel94/Model94_shots.segy.gz'
local_gz = 'Model94_shots.segy.gz'
local_segy = 'Model94_shots.segy'

if not os.path.exists(local_segy):
    print('Downloading BP dataset...')
    with urllib.request.urlopen(bp_url) as resp, open(local_gz, 'wb') as f:
        shutil.copyfileobj(resp, f)
    with gzip.open(local_gz, 'rb') as gz, open(local_segy, 'wb') as out:
        shutil.copyfileobj(gz, out)


In [ ]:

# Scan the SEGY file to find shot locations and offsets
fh, shots, offsets, counts = pysegy.segy_scan(local_segy)
shots = np.array(shots)
offsets = np.array(offsets)
counts = np.array(counts)
print(f'Total shots: {len(shots)}')
print('Samples per trace:', fh.bfh.ns)


In [ ]:

plt.figure(figsize=(6,5))
plt.scatter(shots[:,0], shots[:,1], s=4)
plt.xlabel('SourceX')
plt.ylabel('SourceY')
plt.title('Source Positions')
plt.axis('equal')
plt.show()


In [ ]:

# Plot a few shot gathers using pysegy.read.read_traces
with open(local_segy, 'rb') as f:
    for i in range(3):
        offset = int(offsets[i])
        count = int(counts[i])
        f.seek(offset)
        headers, data = pysegy.read.read_traces(
            f, fh.bfh.ns, count, fh.bfh.DataSampleFormat
        )
        data = np.array(data, dtype=np.float32)
        plt.figure(figsize=(8,4))
        plt.imshow(data[::-1], aspect='auto', cmap='gray')
        plt.title(f'Shot gather {i+1}')
        plt.xlabel('Trace')
        plt.ylabel('Sample')
        plt.show()


In [ ]:

# Receiver positions for the first shot
offset = int(offsets[0])
count = int(counts[0])
with open(local_segy, 'rb') as f:
    f.seek(offset)
    headers, _ = pysegy.read.read_traces(
        f, fh.bfh.ns, count, fh.bfh.DataSampleFormat
    )
rx = [h.GroupX for h in headers]
ry = [h.GroupY for h in headers]
plt.figure(figsize=(6,5))
plt.scatter(rx, ry, s=4)
plt.scatter(shots[0,0], shots[0,1], c='r', marker='x', label='source')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Receiver positions for first shot')
plt.legend()
plt.axis('equal')
plt.show()
